# 8boot/bootstrap.py

In [1]:
import numpy as np, pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import random

import os
os.chdir('../KocPythonFall2021/inclass/6LinMod/')

In [2]:
data = pd.read_stata('TamingGods.dta')

- There is a pdf with explanations of variables
- Lets check if ethnic fractionalization is correlated with religious repression

In [3]:
data = data[['Religion', 'Ethnic', 'polity2_', 'conflict', 'relconflict']]
y = data[['Religion']]
X = pd.DataFrame(data[['Ethnic', 'polity2_', 'conflict', 'relconflict']])
X['constant'] = 1
myFit = sm.OLS(y, X, missing = 'drop').fit()
myFit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Religion   R-squared:                       0.071
Model:                            OLS   Adj. R-squared:                  0.070
Method:                 Least Squares   F-statistic:                     86.70
Date:                Tue, 07 Dec 2021   Prob (F-statistic):           4.44e-71
Time:                        14:39:58   Log-Likelihood:                 215.72
No. Observations:                4530   AIC:                            -421.4
Df Residuals:                    4525   BIC:                            -389.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Ethnic          0.2448      0.014     17.348      0.000       0.217       0.272
polity2_        0.0024      0.001      4.607      0.000       0.001       0.003
conflict       -0.0685      0.010     -7.058      0.000      -0.088      -0.049
relconflict    -0.0110      0.006     -1.729      0.084      -0.023       0.001
constant        0.3287      0.008     42.574      0.000       0.314       0.344
==============================================================================
Omnibus:                      815.491   Durbin-Watson:                   0.063
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              207.649
Skew:                          -0.225   Prob(JB):                     8.12e-46
Kurtosis:                       2.053   Cond. No.                         33.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- Recall the errors are highly problematic

## Bootstrap!
- This is an important concept to understand. Bootstrapping is a general approach to statistical inference based on building a sampling distribution for a statistic by resampling from the data at hand. Bootstrapping offers advantages:
 - The bootstrap is quite general, although there are some cases in which it fails (best example: if there is structure, you will do better modeling the structure, such as temporal correlation, cross-sectional dependencies, etc.; the bootstrap will still generally produce unbiased but inefficient results, and aggregation is not always a good depiction of heterogeneous reality).
 - Because it does not require distributional assumptions (such as normally distributed errors), the bootstrap can provide more accurate inferences when the data are not well behaved or when the sample size is small.
 - It is possible to apply the bootstrap to statistics with sampling distributions that are difficult to derive, even asymptotically.
 - It  is  relatively  simple  to  apply  the  bootstrap  to  complex  data-collection plans (such as stratified and clustered samples).
 - We can even use it to get uncertainty for models (such as NN) that do not lend to uncertainty (teaser for ML)

## algorithm (dataset has N observations):
1. sample N rows / observations WITH REPLACEMENT
2. run your model
3. collect the estimates' coefficients
4. repeat 1-3 several times (about 1000 is usually sufficient)
5. take the mean of the (many) estimates -> point estimate
6. find the std. dev. of the (many) point estimates -> sd of coefficient
7. can also use empirical quantiles for conf ints

### TODO: fill in the functions to run a bootstrap: sampleMod should run a linear regression (omitting NAs) and return parameters; bootstrap should call sampleMod multiple times and return a pd.DataFrame with aggregated estimates, std. dev., and conf int; compareBootOLS should run an OLS and call bootstrap and return a pd.DataFrame with OLS and bootstrapped quantities of interest

In [ ]:
def sampleMod(y, X, seed = 1):
    random.seed(seed)
    #stuff and things; be sure to handle NAs

def bootstrap(y, X, iters = 1000):
    #call sampleMod (be sure to change the seed for each iter in a replicable way)

def compareBootOLS(y, X, iters = 1000):
    #should return a k (no. of params) by 8 (estimates for both approaches, sd of both approaches, .025 and .0975 quantiles of uncertainty) pd.DataFrame

compareBootOLS(y, X) #run on previously defined y and X

# 8boot/bootstrapSolution.py

In [4]:
def sampleMod(y, X, seed = 1):
    random.seed(seed)
    data = pd.DataFrame(np.hstack((y, X))).dropna()
    data = data.sample(n = data.shape[0], replace = True)
    return(sm.OLS(data[0], data.iloc[:, 1:]).fit().params)

def bootstrap(y, X, iters = 1000):
    pars = np.empty((X.shape[1], iters), dtype = float)
    for i in range(iters): pars[:, i] = sampleMod(y, X, i)
    pars = pd.DataFrame(pars)
    betas = pars.apply(np.mean, axis = 1)
    stddevs = pars.apply(np.std, axis = 1)
    CIs = np.quantile(pars, [.025, .975], axis = 1).T
    summaryOut = pd.DataFrame(np.column_stack((np.reshape(betas.values, (len(betas), 1)),
np.reshape(stddevs.values, (len(stddevs), 1)),
CIs)),
columns = ['Estimates', 'Std.Dev.', '0.025', '0.975'])
    return(summaryOut)

def compareBootOLS(y, X, iters = 1000):
    myFit = sm.OLS(y, X, missing = 'drop').fit()
    EstimatesOLS = myFit.params
    StdDevOLS = myFit.bse
    ConfIntOLS = myFit.conf_int()
    summaryOLS = pd.DataFrame(np.column_stack((EstimatesOLS, StdDevOLS, ConfIntOLS)), columns = ['EstimatesOLS', 'Std.Dev.OLS', '0.025OLS', '0.975OLS'])
    summaryBoot = bootstrap(y, X, iters)
    compOut = pd.concat([summaryOLS, summaryBoot], axis = 1)
    compOut = compOut.set_index(ConfIntOLS.index)
    return(compOut)

compareBootOLS(y, X)

,EstimatesOLS,Std.Dev.OLS,0.025OLS,0.975OLS,Estimates,Std.Dev.,0.025,0.975
Ethnic,0.244824,0.014113,0.217157,0.272492,0.244727,0.014241,0.216263,0.271309
polity2_,0.002401,0.000521,0.001379,0.003423,0.002440,0.000518,0.001426,0.003458
conflict,-0.068489,0.009704,-0.087514,-0.049464,-0.068652,0.009850,-0.086457,-0.049146
relconflict,-0.010980,0.006352,-0.023434,0.001473,-0.011061,0.005326,-0.021604,-0.000650
constant,0.328698,0.007721,0.313562,0.343835,0.328574,0.007445,0.314843,0.342708


### TODO: using hw data or any data, run your above function and see if there are discrepancies